In [51]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
import xgboost as xgb
import os
import time
color = sns.color_palette()

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)

In [50]:
# constants
DATA_PATH = '/kaggle/dev/instacart-market-basket-analysis-data/raw_data/'
PREPROCESSED_DATA_PATH = '/kaggle/dev/instacart-market-basket-analysis-data/preprocessed_data/'
SUBMISSIONS_PATH = '/kaggle/dev/instacart-market-basket-analysis-data/submissions/'

In [3]:
# loading the data
aisles = pd.read_csv(os.path.join(DATA_PATH, 'aisles.csv'))
departments = pd.read_csv(os.path.join(DATA_PATH, 'departments.csv'))
order_products_prior = pd.read_csv(os.path.join(DATA_PATH, 'order_products__prior.csv'))
order_products_train = pd.read_csv(os.path.join(DATA_PATH, 'order_products__train.csv'))
orders = pd.read_csv(os.path.join(DATA_PATH, 'orders.csv'))
products = pd.read_csv(os.path.join(DATA_PATH, 'products.csv'))
sample_submission = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))

In [4]:
prod_catalog = pd.merge(products, departments, on='department_id', how='inner')
prod_catalog = pd.merge(prod_catalog, aisles, on='aisle_id', how='inner')
prod_catalog[10000:10010]

,product_id,product_name,aisle_id,department_id,department,aisle
10000,21092,Organic 100% Pure Dark Amber Maple Syrup,29,13,pantry,honeys syrups nectars
10001,21215,Buzzerkeley Honey,29,13,pantry,honeys syrups nectars
10002,21361,Honey Manuka Raw Kfactor 16,29,13,pantry,honeys syrups nectars
10003,21402,Mango Syrup,29,13,pantry,honeys syrups nectars
10004,21933,Clover Honey Squeeze Bottle,29,13,pantry,honeys syrups nectars
10005,22036,Pure 100% Maple Syrup,29,13,pantry,honeys syrups nectars
10006,22107,Sugar Free Classic Caramel Flavoring Syrup,29,13,pantry,honeys syrups nectars
10007,22139,Dark Corn Syrup,29,13,pantry,honeys syrups nectars
10008,22504,Organic Mountain Forest Amber Honey,29,13,pantry,honeys syrups nectars
10009,22554,Sugar Free Low Calorie Artificially Flavored B...,29,13,pantry,honeys syrups nectars


In [5]:
df1 = pd.merge(prod_catalog, order_products_prior, on='product_id', how='inner')
df2 = pd.merge(df1, orders, on='order_id', how='inner')
df2 = df2.sort_values(['user_id', 'order_number', 'order_id', 'add_to_cart_order'])

In [9]:
df2.to_csv(PREPROCESSED_DATA_PATH + 'prior_merged.csv')

In [13]:
df2 = df2[df2['reordered'] == 1]

In [14]:
df2

,product_id,product_name,aisle_id,department_id,department,aisle,order_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
14983311,196,Soda,77,7,beverages,soft drinks,2398795,1,1,1,prior,2,3,7,15.0
14983308,12427,Original Beef Jerky,23,19,snacks,popcorn jerky,2398795,3,1,1,prior,2,3,7,15.0
14983309,26088,Aged White Cheddar Popcorn,23,19,snacks,popcorn jerky,2398795,5,1,1,prior,2,3,7,15.0
14972744,196,Soda,77,7,beverages,soft drinks,473747,1,1,1,prior,3,3,12,21.0
14972741,12427,Original Beef Jerky,23,19,snacks,popcorn jerky,473747,2,1,1,prior,3,3,12,21.0
14972742,10258,Pistachios,117,19,snacks,nuts seeds dried fruit,473747,3,1,1,prior,3,3,12,21.0
14982512,196,Soda,77,7,beverages,soft drinks,2254736,1,1,1,prior,4,4,7,29.0
14982510,12427,Original Beef Jerky,23,19,snacks,popcorn jerky,2254736,2,1,1,prior,4,4,7,29.0
14982511,10258,Pistachios,117,19,snacks,nuts seeds dried fruit,2254736,3,1,1,prior,4,4,7,29.0
14982513,25133,Organic String Cheese,21,16,dairy eggs,packaged cheese,2254736,4,1,1,prior,4,4,7,29.0


In [15]:
df3 = pd.merge(sample_submission, orders, on='order_id', how='left')
df3

,order_id,products,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,17,39276 29259,36855,test,5,6,15,1.0
1,34,39276 29259,35220,test,20,3,11,8.0
2,137,39276 29259,187107,test,9,2,19,30.0
3,182,39276 29259,115892,test,28,0,11,8.0
4,257,39276 29259,35581,test,9,6,23,5.0
5,313,39276 29259,113359,test,31,6,22,7.0
6,353,39276 29259,173814,test,4,4,13,30.0
7,386,39276 29259,55492,test,8,0,15,30.0
8,414,39276 29259,120775,test,18,5,14,8.0
9,418,39276 29259,33565,test,12,0,12,14.0


In [36]:
df4 = df2[['user_id','product_id']].drop_duplicates()
df4['product_id'] = df4['product_id'].astype(str)
df4

,user_id,product_id
14983311,1,196
14983308,1,12427
14983309,1,26088
14972742,1,10258
14982513,1,25133
14982514,1,26405
14972453,1,13176
14973179,1,13032
14982752,1,49235
14982750,1,46149


In [41]:
df5 = df4.groupby('user_id')['product_id'].agg({'products': lambda x:' '.join(x)}).reset_index()
df5

/kaggle/dev/ashish/instacart-market-basket-analysis/insta/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


,user_id,products
0,1,196 12427 26088 10258 25133 26405 13176 13032 ...
1,2,47766 32792 20574 12000 9681 32139 27344 24852...
2,3,21903 17668 32402 39190 47766 9387 16965 28373...
3,4,35469
4,5,43693 24535 11777 26604 40706 21413 13988 8518
5,6,38293 21903
6,7,29871 22963 6361 31683 45066 37999 45628 37602...
7,8,9839 14992 2078 17794 23165 21903 28985 32030 ...
8,9,38159 27966 4957 27973 8834 30252 24341 29594 ...
9,10,46979 27104 47380 16797 28535 30489 47526 9339...


In [49]:
df6 = pd.merge(df3, df5, on='user_id', how='left')
submission_df = df6[['order_id', 'products_y']]
submission_df.columns = ['order_id', 'products']
submission_df

,order_id,products
0,17,21709 47766 38777 13107 21463
1,34,47029 39180 10132 7559 30639 47792 43504 48726...
2,137,25890 38689 23794 43352 41787 24852 35694 4414...
3,182,30007 1244 24009 34243 13629 27104 21903 39275...
4,257,49235 30233 47766 38558 15438 24852 45013 2710...
5,313,25890 49683 13198 21903 16349 46906 12779 4451...
6,353,35561 40688
7,386,24852 38281 47766 39180 33352 45066 15872 4226...
8,414,33198 14947 21709 9755 21230 20392 31730 48988...
9,418,47766 12036 40268 5031 35883 38694 30489 45007...


In [56]:
timestamp = str(int(time.time()))
submission_df.to_csv(SUBMISSIONS_PATH + 'submission-' + timestamp + '.csv', index=False)